<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>
Let's have some fun 
WEEK3

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe like the one shown below:

In [9]:
import pandas as pd
import numpy as np

In [10]:
!pip install lxml
!pip install html5lib
!pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests

In [12]:
website = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(website)
html_doc=r.content
soup = BeautifulSoup(html_doc, 'html.parser')

In [16]:
#examine the html
#print(soup.prettify())

In [13]:
#Grabs the first table in the website and converts to dataframe.
CanadaPostal = pd.read_html(website, header=0)[0]
CanadaPostal.shape

(180, 3)

Interesting that in wiki, it says Toronto - 103 FSAs and there are 180 rows here...

In [38]:
CanadaPostal

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


<h2>3. To create the above dataframe:</h2>

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood <br>
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. <br>
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table. <br>
If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough. <br>
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making. <br>
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [14]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. 
CanadaPostalNA = CanadaPostal[CanadaPostal['Borough']=='Not assigned']
CanadaPostalNA.shape

CanadaPostal.drop(CanadaPostal[CanadaPostal.Borough=='Not assigned'].index, inplace=True)
CanadaPostal.shape
CanadaPostal.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
#More than one neighborhood can exist in one postal code area. 
#For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table. 
CanadaPostal = CanadaPostal.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
print(CanadaPostal.shape)
CanadaPostal.head()

(103, 3)


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
#Nulls
CanadaPostalNNA = CanadaPostal[CanadaPostal['Neighbourhood']=='Not assigned']
print(CanadaPostalNNA.shape)
CanadaPostalNNA.head()

(0, 3)


,Postal Code,Borough,Neighbourhood


In [17]:
#In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
CanadaPostal.shape
#match 103

(103, 3)

<h2>Part 2</h2>
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In [5]:
!pip install geocoder
import geocoder # import geocoder

In [31]:
# initialize your variable to None
#lat_lng_coords = None
#postal_code="M1B"

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]
#print(latitude)

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [20]:
geocodefile = 'http://cocl.us/Geospatial_data'
geo=pd.read_csv(geocodefile,header=0)
geo.sort_values(by='Postal Code', ascending=True)
geo.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
CanadaPostalGeo=CanadaPostal.merge(geo, on='Postal Code')
CanadaPostalGeo.head(5)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h2>Part 3</h2>
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make. <br>
to generate maps to visualize your neighborhoods and how they cluster together. 

In [30]:
tor_count = CanadaPostalGeo['Borough'].str.contains('Toronto').sum()
if tor_count>0:
    print ("There are {m} Toronto".format(m=tor_count))

There are 39 Tors


In [36]:
TorPostal = CanadaPostal[CanadaPostal['Borough'].str.contains('Toronto')]
print(TorPostal.shape)
TorPostal

(39, 3)


,Postal Code,Borough,Neighbourhood
37,M4E,East Toronto,The Beaches
41,M4K,East Toronto,"The Danforth West, Riverdale"
42,M4L,East Toronto,"India Bazaar, The Beaches West"
43,M4M,East Toronto,Studio District
44,M4N,Central Toronto,Lawrence Park
45,M4P,Central Toronto,Davisville North
46,M4R,Central Toronto,"North Toronto West, Lawrence Park"
47,M4S,Central Toronto,Davisville
48,M4T,Central Toronto,"Moore Park, Summerhill East"
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
